In [1]:
%run setting.ipynb

# load data

In [2]:
TQQQ_simulation = pd.read_excel('TQQQ_simulation.xlsx')

In [3]:
QLD_simulation = pd.read_excel('QLD_simulation.xlsx')

In [4]:
NDX = pd.read_csv('^NDX.csv')

In [5]:
GSPC = yf.Ticker('^GSPC').history(
    period='max', 
    interval='1d', 
    actions=False)

GSPC = GSPC.reset_index()
GSPC['id'] = GSPC.index
GSPC['Date'] = pd.to_datetime(GSPC['Date']).dt.date

C:\Users\jimwa\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [6]:
SPXL_simulation = pd.read_excel('SPXL_simulation.xlsx')

# dataframe to table

In [7]:
df_to_table(TQQQ_simulation, 'TQQQ')
df_to_table(QLD_simulation, 'QDL')
df_to_table(NDX, 'NDX')

##

df_to_table(GSPC, 'GSPC')
df_to_table(SPXL_simulation, 'SPXL')

24185

# join all the stocks

In [10]:
sql_to_table(f"""
select a.Date,
a.Close as NDX,
--
b.simulated_QLD,
c.simulated_TQQQ,
d.Close as GSPC,
e.simulated_SPXL
--
from NDX as a 
--
join QDL as b on a.Date = b.Date
join TQQQ as c on a.Date = c.Date
join GSPC as d on a.Date = d.Date
join SPXL as e on a.Date = e.Date
""", 'all_data')

9709

# compare to the first day

In [16]:
sql(f"""

with a as (
select *
from all_data
order by Date asc
limit 1
)

select b.*,
--
b.NDX/a.NDX as NDX_rate,
b.simulated_QLD/a.simulated_QLD as QLD_rate,
b.simulated_TQQQ/a.simulated_TQQQ as TQQQ_rate,
b.GSPC/a.GSPC as GSPC_rate,
b.simulated_SPXL/a.simulated_SPXL as SPXL_rate
--
from all_data as b, a
""").to_excel(f'rate.xlsx', index = False)

# end